In [4]:
from tqdm import tqdm
import cv2
import numpy as np
import os
import shutil
import pandas as pd
import csv

In [120]:
# Get data fro, label.csv
csv_rows = []
with open("../label.csv", 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        csv_rows.append(row)

In [155]:
# Data/
    #train/
        #Fake/
            #imgs/
            #videos/
        #Real/
            #imgs/
            #videos/
    #val
        #Fake/
            #imgs/
            #videos/
        #Real/
            #imgs/
            #videos/
# vides/

# Code
    

In [122]:
# Folder Struture

ORIGINAL_VIDEOS = "../videos/"
TRAIN_FOLDER = "../Data/train/"
VAL_FOLDER = "../Data/val/"
N_VAL_FILES = 200

# Remove all file in data folder
def remove_all_files_in_a_folder(folder_path):
    print(f"Removing file in {folder_path}: {len(os.listdir(folder_path))}")
    for file in os.listdir(folder_path):
        os.remove(folder_path+ file)

remove_all_files_in_a_folder(TRAIN_FOLDER + "Fake/videos/")
remove_all_files_in_a_folder(TRAIN_FOLDER + "Fake/imgs/")
remove_all_files_in_a_folder(TRAIN_FOLDER + "Real/videos/")
remove_all_files_in_a_folder(TRAIN_FOLDER + "Real/imgs/")


remove_all_files_in_a_folder(VAL_FOLDER + "Fake/videos/")
remove_all_files_in_a_folder(VAL_FOLDER + "Fake/imgs/")
remove_all_files_in_a_folder(VAL_FOLDER + "Real/videos/")
remove_all_files_in_a_folder(VAL_FOLDER + "Real/imgs/")

print(f"Removed all file in Data")


Removing file in ../Data/train/Fake/videos/: 0
Removing file in ../Data/train/Fake/imgs/: 0
Removing file in ../Data/train/Real/videos/: 0
Removing file in ../Data/train/Real/imgs/: 0
Removing file in ../Data/val/Fake/videos/: 0
Removing file in ../Data/val/Fake/imgs/: 0
Removing file in ../Data/val/Real/videos/: 0
Removing file in ../Data/val/Real/imgs/: 0
Removed all file in Data


In [123]:
# train videos data
process_rows = csv_rows[1: -N_VAL_FILES]
with tqdm(total=len(process_rows), desc="Moving files: ", bar_format="{l_bar}{bar}        [ time left: {remaining} ]") as pbar:
    for row in process_rows:
        filename = row[0]
        label = row[1]
        if label == "0":
            # move to Fake folder
            src = ORIGINAL_VIDEOS + filename
            dst = TRAIN_FOLDER + "Fake/videos/" + filename
            shutil.copyfile(src, dst)
        else:
            # move to Real folder
            src = ORIGINAL_VIDEOS + filename
            dst = TRAIN_FOLDER + "Real/videos/" + filename
            shutil.copyfile(src, dst)
        pbar.update(1)
print(f"Total file moved: {len(process_rows)}")
print(f"Total file in Fake: {len(os.listdir(TRAIN_FOLDER + 'Fake/videos/'))}")
print(f"Total file in Real: {len(os.listdir(TRAIN_FOLDER + 'Real/videos/'))}")

Moving files: 100%|████████████████████████████████████████████████████████████████████████        [ time left: 00:00 ]

Total file moved: 968
Total file in Fake: 477
Total file in Real: 491


In [124]:
# val videos data 
process_rows = csv_rows[-N_VAL_FILES:]
with tqdm(total=len(process_rows), desc="Moving files: ", bar_format="{l_bar}{bar}        [ time left: {remaining} ]") as pbar:
    for row in csv_rows[-200:]:
        filename = row[0]
        label = row[1]
        if label == "0":
            # move to Fake folder
            src = ORIGINAL_VIDEOS + filename
            dst = VAL_FOLDER + "Fake/videos/" + filename
            shutil.copyfile(src, dst)
        else:
            # move to Real folder
            src = ORIGINAL_VIDEOS + filename
            dst = VAL_FOLDER + "Real/videos/" + filename
            shutil.copyfile(src, dst)
        pbar.update(1)
        
print(f"Total file moved: {len(process_rows)}")
print(f"Total file in Fake: {len(os.listdir(VAL_FOLDER + 'Fake/videos/'))}")
print(f"Total file in Real: {len(os.listdir(VAL_FOLDER + 'Real/videos/'))}")

Moving files: 100%|████████████████████████████████████████████████████████████████████████        [ time left: 00:00 ]

Total file moved: 200
Total file in Fake: 93
Total file in Real: 107


# Convert Videos to imgs

In [125]:
def split_imgs_from_videos(video_path, imgs_path, filename, rate = 1):
    video = cv2.VideoCapture(video_path)
    # frame
    currentframe = 0
    while True:
        ret,frame = video.read()
        if ret: # if video is still left continue creating images
            if currentframe % rate == 0:
                name = imgs_path + filename +"_"+ str(currentframe) + '.png'
                cv2.imwrite(name, frame)
            currentframe += 1
        else:
            break

In [126]:
def  split_imgs(folder_path, rate):
    videos = os.listdir(folder_path+"videos/")
    print(f"Total video in folder {folder_path} : {len(videos)}")
    with tqdm(total=len(videos), desc="Spliting imgs: ", bar_format="{l_bar}{bar}        [ time left: {remaining} ]") as pbar:
        for video in videos:
            filename = video[:-4]
            video_path = folder_path + "videos/" + video
            imgs_path = folder_path + "imgs/"
            split_imgs_from_videos(video_path, imgs_path, filename, rate)
            pbar.update(1)
    print(f"Total new imgs splited: {len(os.listdir(folder_path+'imgs/'))}")
       
        
            

In [127]:
split_imgs(TRAIN_FOLDER + "Fake/", 100)

Total video in folder ../Data/train/Fake/ : 477


Spliting imgs: 100%|███████████████████████████████████████████████████████████████████████        [ time left: 00:00 ]

Total new imgs splited: 954


In [128]:
split_imgs(TRAIN_FOLDER + "Real/", 100)

Total video in folder ../Data/train/Real/ : 491


Spliting imgs: 100%|███████████████████████████████████████████████████████████████████████        [ time left: 00:00 ]

Total new imgs splited: 972


In [129]:
split_imgs(VAL_FOLDER + "Real/", 100)

Total video in folder ../Data/val/Real/ : 107


Spliting imgs: 100%|███████████████████████████████████████████████████████████████████████        [ time left: 00:00 ]

Total new imgs splited: 212


In [130]:
split_imgs(VAL_FOLDER + "Fake/", 100)

Total video in folder ../Data/val/Fake/ : 93


Spliting imgs: 100%|███████████████████████████████████████████████████████████████████████        [ time left: 00:00 ]

Total new imgs splited: 186


# Imgs processing

In [151]:
def sharpening(img_path, filename):
    img = cv2.imread(img_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite("../Data/tmp/"+filename, img_gray)
    kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
    image_sharp = cv2.filter2D(src=img_gray, ddepth=-1, kernel=kernel)
    cv2.imwrite("../Data/train/Fake/img_processed/"+filename, image_sharp)

In [152]:
files = os.listdir("../Data/train/Fake/imgs")

In [154]:
for file in files: 
    sharpening("../Data/train/Fake/imgs/"+ file, file)